In [1]:

#!/usr/bin/env python
# coding: utf-8

# # CODE

# In[1]:


import os

import scanpy as sc
import anndata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

from scipy import sparse
from matplotlib import rcParams
rcParams['pdf.fonttype'] = 42 # enables correct plotting of text
rcParams['figure.figsize'] = (7,7)
import seaborn as sns
from scipy import sparse

import random
import tensorflow as tf

seed = 42


random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

palette = {'CNT':'#1f77b4',
 'DCT1':'#ff7f0e',
 'DCT2':'#279e68',
 'TL':'#279e68',
 'DCT':'#279e68',
 'ENDO':'#d62728',
 'FIB':'#aa40fc',
 'ICA':'#8c564b',
 'ICB':'#e377c2',
 'LEUK':'#b5bd61',
 'MES_FIB':'#17becf',
 'MES':'#17becf',
 'PC':'#aec7e8',
 'PEC':'#ffbb78',
 'PODO':'#98df8a',
 'PT':'#ff9896',
 'PT_VCAM1':'#c5b0d5',
 'TAL':'#c49c94',
 'Unclassified':'#808080',
 'Unknown':'#000000',
 'Low_Quality_RNA':'#808080'}


# In[3]:


import sys 
from deepscore import deepscore
from deepscore import marker_analysis
import pickle


# In[4]:
cell_type = 'subclass.l3'


# # FINE GRAINED LABEL TRANSFER PER COMPARTMENT

# ## Preprocessing

# In[5]:


ref_py = sc.read('/home/macera/Documentos/CZI/MANUSCRIPT_PREP/HORIZONTAL_RNA/objects/local.h5ad')  # LOAD the HCA Atlas
ref_py = ref_py.raw.to_adata()
ref_py.layers['counts'] = ref_py.X.copy()

ref_py.var['ENSG'] = ref_py.var.index.copy()
ref_py.var.index = ref_py.var['feature_name'].copy()

cortex_celltypes_l1= ['DCT',
 'PEC',
 'CNT',
 'POD',
 'PT',
 'IC',
 'IMM',
 'NEU',
 'VSM/P',
 'TAL',
 'EC',
 'FIB',
 'PC']

cortex_celltypes_l3= ['DCT1',
 'DCT2',
 'IC-B',
 'B',
 'MD',
 'CNT-IC-A',
 'MC',
 'CNT',
 'PT-S1/2',
 'CCD-IC-A',
 'PEC',
 'pDC',
 'EC-GC',
 'POD',
 'VSMC',
 'aPT',
 'CNT-PC',
 'FIB',
 'cDC',
 'REN',
 'EC-PTC',
 'T',
 'PT-S3',
 'ncMON',
 'NKC/T',
 'aTAL1',
 'C-TAL',
 'PL',
 'CCD-PC',
 'SC/NEU',
 'EC-LYM',
 'MDC',
 'N',
 'MAST',
 'aTAL2',
 'aFIB',
 'MYOF',
 'MAC-M2',
 'EC-AEA',
 'VSMC/P']



ref_py = ref_py[ref_py.obs['subclass.l1'].isin(cortex_celltypes_l1)]

ref_py = ref_py[ref_py.obs['subclass.l3'].isin(cortex_celltypes_l3)]

ref_py = ref_py[ref_py.obs['condition.long'].isin(['Normal Reference'])]


ref_py = ref_py[ref_py.obs['state.l2'].isin(['reference','adaptive - epi','adaptive - str'])].copy()


compute = False
cell_type = 'subclass.l3'

# for i in ref_py.obs['subclass.l1'].unique().tolist():
#     a = i.replace('/','_')
#     # if os.path.exists(f'objects/reference_subset/{a}.h5ad') == False:
#     ref_py[ref_py.obs['subclass.l1'].isin([i])].write(f'objects/reference_subset_clean/{a}.h5ad', compression='gzip')




2025-09-02 16:05:33.428592: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-02 16:05:33.716746: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-02 16:05:33.718152: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-02 16:05:34.752959: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:

import random
import tensorflow as tf

seed = 42


random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

In [3]:
cell_type_list = ['IMM', 'DCT', 'PT', 'CNT', 'TAL', 'IC', 'EC', 'FIB', 'VSM/P']

In [ ]:

if compute:
    for i in cell_type_list:
        a = i.replace('/','_')
        ref_py = sc.read(f'objects/reference_subset_clean/{a}.h5ad')

        markers_filename= f'HCA_{a}_l3'
        print(a)

        sc.pp.normalize_total(ref_py, target_sum=1e4)
        sc.pp.log1p(ref_py)
    # Identify differentially expressed genes between cell types
        sc.tl.rank_genes_groups(ref_py, cell_type, method='wilcoxon', use_raw=False)
        ranked_genes_populations = ref_py.uns['rank_genes_groups'].copy()
        with open(f'markers_ds/{markers_filename}.pickle', 'wb') as handle:
            pickle.dump(ranked_genes_populations, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [6]:
mod = 'RECK'

adata = sc.read(f'/home/macera/Documentos/CZI/MANUSCRIPT_PREP/REVIEWS/external_data/reck/rna_ready.h5ad', compression='gzip')
# adata.X = adata.layers['counts'].copy()
# adata = adata[adata.obs['batch'].isin([mod])].copy()
ds_hca = pd.read_csv(f'csv/Deepscore_HCA_l1_{mod}_CLEAN.csv', index_col=0)
adata.obs['Deepscore_HCA_l1'] = ds_hca['Deepscore_HCA'].astype('category')
adata.obs['Deepscore_HCA_l1_score'] = ds_hca['Deepscore_HCA_score'].astype('category')

# In[33]:
try:
    os.mkdir(f'objects/{mod}_subset/')
except:
    print('Directory for subset saving already exists')


for i in adata.obs['Deepscore_HCA_l1'].unique():
    print(i)
    a = i.replace('/','_')
    # if os.path.exists(f'objects/reference_subset/{a}.h5ad') == False:
    adata[adata.obs['Deepscore_HCA_l1'].isin([i])].write(f'objects/{mod}_subset/{a}.h5ad', compression='gzip')



TAL
IMM
EC
PT
IC
CNT
PC
DCT
PEC
VSM/P
POD
FIB
NEU


In [9]:
cell_type_list

['IMM', 'DCT', 'PT', 'CNT', 'TAL', 'IC', 'EC', 'FIB', 'VSM/P']

In [8]:

n_markers = 300
cell_type = 'subclass.l3'

## SET PARAMETERS

# ## Train & Inference

for i in cell_type_list:
    a = i.replace('/','_')
    if os.path.exists(f'csv/Deepscore_HCA_l3_{mod}_{a}_CLEAN.csv'):
        print(f'{mod} {a} already exists!')
    else:
        ref_py = sc.read(f'/home/macera/Documentos/CZI/MANUSCRIPT_PREP/DEEPSCORE/objects/reference_subset_clean/{a}.h5ad')
        adata = sc.read(f'objects/{mod}_subset/{a}.h5ad', compression='gzip')

        sc.pp.normalize_total(adata, target_sum=1e4)
        sc.pp.log1p(adata)

        markers_filename = f'HCA_{a}_l3'       
        # Identify differentially expressed genes between cell types

        with open(f'markers_ds/{markers_filename}.pickle', 'rb') as handle:
            ranked_genes_populations = pickle.load(handle) 

            # Step 1: Create a dictionary to store markers for each subset
        subset_markers_dict = {}

        # Step 2: Store markers for each subset
        for subset in ref_py.obs[cell_type].unique():
            subset_markers = ranked_genes_populations['names'][subset]
            subset_markers = [gene for gene in subset_markers if gene in adata.var.index]
            subset_markers_dict[subset] = set(subset_markers[:n_markers+100])

        # Step 3: Identify overlapping markers
        overlapping_markers = set()
        for subset, markers in subset_markers_dict.items():
            for other_subset, other_markers in subset_markers_dict.items():
                if subset != other_subset:
                    overlapping_markers.update(markers.intersection(other_markers))

        # Step 4: Select markers for each subset, excluding overlapping markers
        marker_dict = {}
        for subset, markers in subset_markers_dict.items():
            unique_markers = [marker for marker in markers if marker not in overlapping_markers]
            marker_dict[subset] = unique_markers[:n_markers]  # Select up to 200 unique markers
            # print(subset,len(unique_markers[:n_markers]))
        selected_markers = [marker for subset in marker_dict for marker in marker_dict[subset]]

        # print(selected_markers)

        sc.pp.scale(ref_py)
        sc.pp.scale(adata) 


        # Subset the data to the selected markers
        ref_py = ref_py[:, list(selected_markers)].copy()
        adata = adata[:, list(selected_markers)].copy()



        ref_py.obs[cell_type] = ref_py.obs[cell_type].tolist()
        len(ref_py.obs[cell_type].unique())




        import tensorflow as tf



        def scheduler(epoch, lr):
            if epoch < 10:
                return lr
            else:
                return lr * tf.math.exp(-0.1)


        n_feat = ref_py.shape[1]
        n_labs = len(ref_py.obs[cell_type].unique())

        ds = deepscore.DeepScore(hidden_nodes=[512, 256],
                        n_features=n_feat, 
                        n_labels=n_labs,
                        epochs=30,
                        batch_size=128, 
                        activation="relu", 
                        dropout=True, 
                        dropout_rate=0.3,
                        batchnorm=True, 
                        lr=0.001,
                        weight_reg=True)



        os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"

        import sklearn

        ds.set_reference(ref_py, label_by=cell_type, test_prop=0.1)

        ds.train(earlystopping=True, patience=10, lr_scheduler=scheduler,)

        prob_df, adata = ds.annotate(adata, pred_key='Deepscore_HCA',Unclassified = False,return_pred_matrix=True)


        adata.obs[['Deepscore_HCA','Deepscore_HCA_score']].to_csv(f'csv/Deepscore_HCA_l3_{mod}_{a}_CLEAN.csv')

        prob_df.to_csv(f'csv/prob_matrix/Deepscore_HCA_l3_{mod}_{a}_CLEAN.csv')




Model: "deepscore"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (Batch  (None, 1779)              7116      
 Normalization)                                                  
                                                                 
 dense512 (Dense)            (None, 512)               911360    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 batch_normalization_1 (Bat  (None, 512)               2048      
 chNormalization)                                                
                                                                 
 dense256 (Dense)            (None, 256)               131328    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0 